In [44]:
import pandas as pd
import re
import json
import numpy as np

In [45]:
with open('aggregate_results.json') as f:
  aggregate_results = json.load(f)

In [60]:
# General functions used to develop extractors

def general_splitter(x, patterns):
    new_string = str(x)
    first_pass = new_string.split(patterns[0], 1)[1]
    second_pass = first_pass.split(patterns[1], 1)[0]
    return(second_pass)

def pattern_sequencer(x, patterns, potential_patterns):
    for p in range(0, len(potential_patterns)):
            if str(x).find(potential_patterns[p][0]) != -1:
                patterns = potential_patterns[p]
                break
            else:
                patterns = ["No Matching Pattern", ""]
    return(patterns)
    
def general_extractor(x, patterns, potential_patterns, keyword):
    if str(x).find(keyword) != -1:
        new_patterns = pattern_sequencer(x, patterns, potential_patterns)
        if new_patterns[0] != "No Matching Pattern":
            return(general_splitter(x, new_patterns))
        else:
            return("Bad Pattern")
    else:
        return("Can't find keyword")
    
def conditional_extractor(Date, Firm, Status, x):
    if Date == "Can't find keyword" and Firm == "Can't find keyword" and Status == "Can't find keyword":
        patterns = ["View Decision</header>\n<div class=\"field-items-wrapper\">\n<div class=\"field__item\"><h2>", "</h2"]
        try:
            return(general_splitter(x, patterns))
        except:
            return("Failed")
            
    else:
        return("")
    
def date_fixer(catch):
    if catch != "" and catch != "Failed":
        try: 
            new_date = re.findall(r"\w+\s\d{2}\,\s\d{4}", catch)[0]
        except:
            new_date = ""
    else:
        new_date = ""
    return(new_date)

def firm_fixer(catch):
    if catch != "" and catch != "Failed":
        try: 
            new_firm = re.split(r"\,", catch)[0]
        except:
            new_firm = ""
    else:
        new_firm = ""
        
    return(new_firm)

In [47]:
firm_patterns = ["Matter of:\xa0</strong> ", "</p>"]
potential_firm_patterns = [["Matter of: </strong><a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                           ["Matter of:\xa0</strong> ", "</p>\n<p><strong>File:"],
                          ["Matter of:</strong> ", "</p>"],
                          ["Matter of:\xa0 </strong>", "</p>"],
                          ["Matter of:</strong>", "</p>"],
                          ["Matter of: \xa0</strong> ","</p>"],
                          ["Matter of:\xa0\xa0\xa0\xa0\xa0 </strong>", "</p>"],
                          ["Matter of: </strong>", "</p>"],
                          ["<p class=\"MatterOf\">\n\t\tMatter of: ", "</p>"],
                          ["Matter of:      </strong>", "</p>"],
                          ["<p class=\"MsoBodyText\"> </p><p class=\"MatterOf\">Matter of: ", "</p>"],
                          ["<p class=\"MatterOf\"><strong>Matter of:   </strong>", "</p>"],
                          ["</p><p><strong>Matter of:   </strong>", "</p>"],
                          ["<p class=\"MatterOf\">Matter of: <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:  <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:  </strong><a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of: <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a></strong>", "<a"],
                          ["Matter of: <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of: <a id=\"OLE_LINK3\" name=\"OLE_LINK3\"> </a> <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"> </a>", "<a"],
                          ["Matter of: <a id=\"MatterofBegin\" name=\"MatterofBegin\"></a>", "<a"],
                          ["Matter of: <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"> </a>", "<a"],
                          ["Matter of:  </strong> <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:    <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:   </strong> <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:<a id=\"MatterOfEnd\" name=\"MatterOfEnd\"></a>", "<a"],
                          ["<p class=\"MatterOf\">Matter of: ", "<a"],
                          ["<p>\n\tMatter of: ", "<a"],
                          ["Matter of:   <a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"],
                          ["Matter of:<a id=\"MatterOfBegin\" name=\"MatterOfBegin\"></a>", "<a"]]

date_patterns = ["<p><strong>Date:\xa0</strong> ", "</p>"]                        
potential_date_patterns = [["<p><strong>Date:</strong> <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                           ["<p><strong>Date:</strong>\xa0 ", "</p>"],
                           ["<p><strong>Date:\xa0</strong> ", "</p>"],
                          ["<p>\n<strong>Date:</strong>\xa0 ", "</p>"],
                          ["<p><strong>Date:</strong> ", "</p>"],
                          ["Date:</strong>", "</p>"],
                          ["Date:\xa0 </strong>", "</p>"],
                          ["Date:\xa0</strong> ", "</p>"],
                          ["Date:\xa0 \xa0</strong> ", "</p>"],
                          ["Date:</strong>\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0", "</p>"],
                          ["Date:</strong> ", "</p>"],
                          ["Date:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 </strong>", "</p>"],
                          ["Date: </strong>", "</p>"],
                          ["<p class=\"Date\">\n\t\tDate: ", "</p>"],
                          ["<p class=\"Date\">Date: ", "</p>"],
                           ["Date:</strong> <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date: <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date:  <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date: <a id=\"DateBegin\" name=\"DateBegin\"> </a>", "<a"],
                          ["Date:<a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date:               <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date:  <a id=\"DateBegin\" name=\"DateBegin\"> </a>", "<a"],
                          ["<p class=\"MsoBodyText\"></p><p>Date: ", "</p>"],
                          ["<p>\n\tDate: ", "</p"],
                          ["Date:   <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"],
                          ["Date:            <a id=\"DateBegin\" name=\"DateBegin\"></a>", "<a"]]

paragraph_patterns = ["</p>\n<div>\n<p>DECISION</p>\n</div>\n<p>" , "</p>\n"]
potential_paragraph_patterns = [["DECISION</p> </div> <p class=\"MsoBodyText\"></p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                                ["</p>\n<div>\n<p>DECISION</p>\n</div>\n<p>" , "</p>\n"],
                                ["<div>\n<p><strong>DECISION</strong></p>\n</div>\n<p>", "</p>"],
                               ["DECISION</p>\n<p>", "</p>"],
                               ["DECISION</p>\n</div>\n<p>\n\t\t", "</p>"],
                                ["DECISION</strong></p>\n<p>\n\t\t", "</p>\n<p>\n\t\t"],
                               ["DECISION</p>\n</div>\n<p>\n\t\t", "\xa0</p>\n<p>\n\t\t"],
                               ["DECISION</strong></div>\n<p>", "</p>"],
                               ["DECISION</strong></p>\n<p>", "</p>"],
                               ["DECISION</p>\n</div>\n<p>", "</p>"],
                               ["DECISION\xa0 </strong></p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION\xa0</p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p>\n\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoBodyText\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoNormal\">\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p class=\"MsoNormal\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Date\">\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n<p class=\"MsoNormal\">\n\t\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoBodyText\">\n\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MatterOf\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Default\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Decision1\">\n\t\t", "</p>"],
                               ["DECISION</p>\n<p class=\"Decision1\">\n\t\t\t", "</p>"],
                               ["DECISION</strong></p>\n<p class=\"MsoBodyText\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"matterof0\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoNormal\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"MsoNormal\">", "</p>"],
                                ["DECISION</p></div><p class=\"MsoBodyTextCxSpFirst\"> </p><p class=\"MsoBodyTextCxSpMiddle\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"MatterOf\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoNormal\"> </p><p class=\"MsoNormal\">", "</p>"],
                               ["DECISION</strong></p></div><p class=\"MsoBodyText\"> </p><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</p><p class=\"Decision1\"> </p><p class=\"Decision1\">", "</p>"],
                               ["DECISION</strong></p><p class=\"MsoBodyText\"> </p><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</p></div><p class=\"MatterOf\"> </p><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</strong></p><p class=\"MsoNormal\"> </p></div><p class=\"MsoNormal\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"Style0\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"Default\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</p></div><p class=\"MatterOf\"> </p><p class=\"MatterOf\">", "</p>"],
                               ["DECISION</p><p> </p><p>", "</p>"],
                               ["DECISION</p></div><p> </p><p>", "</p>"],
                               ["DECISION</p><p class=\"Decision1\"> </p></div><p class=\"MsoNormal\">NAS/", "</p>"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p class=\"MsoBodyText\"> </p></div><p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MatterOf\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "</p"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MatterOf\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> <p class=\"Decision1\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> <p class=\"Decision1\"></p> <p class=\"Decision1\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> <p class=\"Decision1\"></p> <p class=\"Decision1\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\">", "<a"],
                               ["DECISION</p> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION <p class=\"MsoBodyText\"></p> </div> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\"> </p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</strong></p> </div> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoBodyText\">", "<a"],
                               ["DECISION</p> </div> <p class=\"MatterOf\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> <p class=\"MsoBodyText\"> </p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> <p class=\"MsoBodyText\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"Decision1\"></p> <p class=\"Decision1\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>V", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoBodyText\"></p> <p class=\"Decision\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"> </a>", "<a"],
                               ["DECISION</p> <p class=\"MsoBodyText\"></p> <p class=\"Decision1\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> <p class=\"MsoBodyText\"></p> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION<a id=\"DigestBegin\" name=\"DigestBegin\"></a></p> </div> <p></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p><p></p> </div> <p class=\"MsoNormal\"></p> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</h3> <p> ", "</p>"],
                               ["DECISION</strong> <br/><br/><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a> <a id=\"Paragraph1End\" name=\"Paragraph1End\"></a>", "<a"],
                               ["DECISION</strong><br/><br/><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p> </p> <p><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoNormal\"></p> <p class=\"MsoNormal\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p> </p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"> </a>", "<a"],
                               ["DECISION </p> <p> </p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p></p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MsoNormal\"></p> <p class=\"MsoBodyText\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</strong> <br/><p><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</p> </div> <p class=\"MatterOf\"><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION </p> <p> </p> <p> <a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</strong><br/><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a> ", "<a"],
                               ["DECISION</strong><br/></p> <p><a id=\"Paragraph1Begin\" name=\"Paragraph1Begin\"></a>", "<a"],
                               ["DECISION</h3> <p> ", "</p"]]

status_patterns = ['<div class="status highlighted-status">\n', '.\n      </div>']
potential_status_patterns = [['<div class="status highlighted-status">\n', '.\n      </div>']]

highlights_patterns = ["<html><body><p>\n\t", "</p>"]
potential_highlights_patterns = [["<html><body><p>", "</p>"], 
                                 ["<html><body><p>\n\t", "</p>"],
                                ["<html><body><p align=\"LEFT\">", "</p>"],
                                ["<html><body><div>\n\t", "</p>"]]

In [281]:
good_df = pd.DataFrame()

for d in range(0, 5416):
    document = aggregate_results[str(d)]['document']
    stat = aggregate_results[str(d)]['status']
    high = aggregate_results[str(d)]['highlights']
    firm = general_extractor(document, firm_patterns, potential_firm_patterns, keyword = "Matter of:")
    date = general_extractor(document, date_patterns, potential_date_patterns, keyword = "Date:")
    status = general_extractor(stat, status_patterns, potential_status_patterns, keyword = "status")
    decision_paragraph = general_extractor(document, paragraph_patterns, potential_paragraph_patterns, keyword = "DECISION")
    highlights = general_extractor(high, highlights_patterns, potential_highlights_patterns, keyword = "Highlights")
    catch = conditional_extractor(date, firm, status, document)
    new_date = date_fixer(catch)
    new_firm = firm_fixer(catch)
    series = {"Firm" :str(firm), 
              "Firm_New" : str(new_firm),
              "Date" : str(date), 
              "Date_new" : str(new_date),
              "Status" : str(status), 
              "Decision_Paragraph" : str(decision_paragraph), 
              "Highlights_Paragraph" : str(highlights), 
              "Catch": str(catch), 
              "URL" : aggregate_results[str(d)]['url']}
    good_df = good_df.append(series, ignore_index = True)

In [49]:
date_bad_pattern = good_df.Date == "Bad Pattern"
firm_bad_pattern = good_df.Firm == "Bad Pattern"
status_bad_pattern = good_df.Status == "Bad Pattern"
decision_bad_pattern = good_df.Decision_Paragraph == "Bad Pattern"
highlights_bad_pattern = good_df.Highlights_Paragraph == "Bad Pattern"

check_df = good_df[decision_bad_pattern | firm_bad_pattern | date_bad_pattern | decision_bad_pattern | highlights_bad_pattern]
check_df

,Date,Decision_Paragraph,Firm,Highlights_Paragraph,Status,URL
4757,"March 22, 2005",Bad Pattern,Park Tower Management Ltd.,Park Tower Management Ltd. protests the award ...,We deny the protest,https://www.gao.gov/products/b-295589%2cb-2955...
5104,Bad Pattern,<br/><p>We sustain RAM's protest and deny COB...,"<a id=""MatterOfBegin"" name=""MatterOfBegin""></a...","Research Analysis &amp; Maintenance, Inc. (RAM...",We sustain RAM's protest and deny COBR...,https://www.gao.gov/products/b-292587.4%2cb-29...
5401,Can't find keyword,Bad Pattern,Can't find keyword,\n\tA firm protested a Department of Transport...,Can't find keyword,https://www.gao.gov/products/b-290163%2cb-2901...
5413,Bad Pattern,"Digital Systems Group, Inc. (DSG) protests the...",Bad Pattern,"\n\tDigital Systems Group, Inc. (DSG) protests...",Can't find keyword,https://www.gao.gov/products/b-286931%2cb-2869...
5414,Bad Pattern,Floro &amp; Associates protests task order No....,Bad Pattern,A firm protested an Army task order for manage...,Can't find keyword,https://www.gao.gov/products/b-285451.3%2cb-28...


In [282]:
good_df.to_excel("Good_Output.xlsx")

In [268]:
p = "Can't find keyword"

def checker(column, pattern):
    return(column == pattern)

check_df2 = good_df[checker(good_df.Date, p) | checker(good_df.Firm, p)]

In [ ]:
sum(checker(good_df.Date, p))
sum(checker(good_df.Firm, p))
sum(checker(good_df.Status, p))
sum(checker(good_df.Decision_Paragraph, p))

In [181]:
re.findall(r"\w+\s\d{2}\,\s\d{4}", check_df2.loc[5409,'Catch'])[0]

'December 17, 2001'

In [274]:
for index, row in good_df.iterrows():
    output_fixer(row)